In [1]:
# Imports
# Standard Libraries of Python
import itertools
from collections import Counter
from typing import Tuple, Union
from decimal import Decimal, ROUND_HALF_UP, getcontext
from datetime import datetime
getcontext().prec = 5

# Dependencies
import pandas as pd
import numpy as np
from pandas import DataFrame, Index

# Libraries made for this Proyect
from backend.src.parse import Criteria
# from backend.src import pick_numbers
file_route = 'database/files/'

In [ ]:
def clean_df_skips(df: DataFrame,columns_id,name) -> DataFrame:
    df.columns = columns_id
    df.columns.name = name
    df.index.name = 'Draws'
    return df

def combination_df(database,low_high_counts,odd_even_counts):
    COMBINATIONS = [(3,2), (2,3), (1,4), (4,1), (0,5), (5,0)]
    draws = set(range(0,len(database)))
    columns_id = ['3/2', '2/3', '1/4', '4/1', '0/5', '5/0']
    
    low_high = {}
    odd_even = {}
    for i in draws:
        counts_l_h = {}
        counts_o_e = {}
        for combination in COMBINATIONS:
            count_l_h = sum([1 for j in range(i-9,i+1) if combination[0] == low_high_counts[j][0] and combination[1] == low_high_counts[j][1]])
            counts_l_h[combination] = count_l_h
            count_o_e = sum([1 for j in range(i-9,i+1) if combination[0] == odd_even_counts[j][0] and combination[1] == odd_even_counts[j][1]])
            counts_o_e[combination] = count_o_e
        low_high[i] = counts_l_h
        odd_even[i] = counts_o_e
    
    low_high = clean_df_skips(pd.DataFrame.from_dict(low_high, orient='index'), columns_id, 'L/H')
    odd_even = clean_df_skips(pd.DataFrame.from_dict(odd_even, orient='index'), columns_id, 'O/E')
    return low_high, odd_even

def count_100_combinations(df, columns, combinations, name):
    count_dic = {i: {key: 0 for key in combinations} for i in range(1, len(df) - 99)}
    columns_id = ['3/2', '2/3', '1/4', '4/1', '0/5', '5/0']
    for i, _ in enumerate(range(1, len(df) - 99)):
        df_slice = df.iloc[i:i+100]
        counts = [df_slice[(df_slice[columns[0]] == combination[0]) & (df_slice[columns[1]] == combination[1])][columns[0]].count() for combination in combinations]
        count_dic[i+1] = dict(zip(combinations, counts))
    df = clean_df_skips(pd.DataFrame.from_dict(count_dic, orient='index'), columns_id, name)
    return df

In [ ]:
# Test of Tomorrow Numbers
# euromillions = Criteria()

# euromillions.groups_info()
# euromillions.apply_transformation()
# euromillions.count_skips()
# euromillions.skips_for_last_12_draws()
# euromillions.get_natural_rotations()
# euromillions.numbers_clasification()

# euromillions.year_criterion()
# euromillions.rotation_criterion()
# euromillions.position_criterion()
# euromillions.group_criterion()
# euromillions.numbers_of_tomorrow()

# lotto = pick_numbers.Selection(euromillions)
# lotto.first_number()
# lotto.suggested_numbers()

# tickets = itertools.combinations(lotto._selected_numbers,5)

# ticket = []
# for combination in tickets:
#     ticket.append(combination)
#     print(combination)

In [2]:
# Test of Tomorrow Stars

euromillions = Criteria(is_star=True)

euromillions.apply_transformation(is_star=True)
euromillions.count_skips(is_star=True)
euromillions.get_natural_rotations(is_star=True)
print(euromillions.year_history)

print(euromillions.aprox_rotation)
print(euromillions.exact_rotation)

      1   2   3   4   5   6   7   8   9   10  11  12
2016  18  28  15  16  18  16  16  19  20  21  20   3
2017  14  19  26  19  21  13  14  17  23  16  12  14
2018  15  18  20  21  11  16  13  19  15  11  21  28
2019  15  27  17  17  16  26  20  17  10  16  17  12
2020  16  20  20  18  15  21  17  19  16  13  15  18
2021  14  23  20  13  10  22  21  19  18  17  18  15
2022  18  17  26  12  13  20  13  16  14  14  25  20
2023  10  21  21  12  18  17  20  14  20  24  17  14
2024   1   0   2   0   0   0   0   1   1   1   0   2
    hits  average_of_numbers total_average minimal_hits_needed difference
1    121            0.072110      0.084333              141.51     -20.51
2    173            0.103099      0.084333              141.51      31.49
3    167            0.099523      0.084333              141.51      25.49
4    128            0.076281      0.084333              141.51     -13.51
5    122            0.072706      0.084333              141.51     -19.51
6    151            0.0899

In [ ]:
euromillions.counts.max()

In [ ]:
euromillions.df.loc[euromillions.df['dates'] == datetime(2016,9,27)]

In [15]:
stars_group = {
    'A': [1,3,5],
    'B': [2,4,6],
    'C': [7,9,11],
    'D': [8,10,12]
}

In [18]:
euromillions.df

,dates,draw,nro1,nro2,nro3,nro4,nro5,star_1,star_2
0,2004-02-13,1,16,29,32,36,41,7,9
1,2004-02-20,2,7,13,39,47,50,2,5
2,2004-02-27,3,14,18,19,31,37,4,5
3,2004-03-05,4,4,7,33,37,39,1,5
4,2004-03-12,5,15,24,28,44,47,4,5
...,...,...,...,...,...,...,...,...,...
1697,2023-12-29,1698,2,3,19,36,37,6,9
1698,2024-01-02,1699,7,15,18,46,49,10,12
1699,2024-01-05,1700,4,7,18,39,50,3,8
1700,2024-01-09,1701,2,9,12,39,40,1,3


In [20]:
letter_init = euromillions.df['star_1'].apply(lambda x: next((key for key,values in stars_group.items() if x in values),None))

In [23]:
letter_init

0       C
1       B
2       B
3       A
4       B
       ..
1697    B
1698    D
1699    A
1700    A
1701    C
Name: star_1, Length: 1702, dtype: object